In [8]:
#list of unordered pairs of elements in {1,...,n}
def pairs(n): return flatten([[[i+1,j+1] for j in range(i+1,n)] for i in range(n-1)],max_level=1)

#sigma acts on an unordered pair
def act(sigma, p): return sorted([sigma(p[0]),sigma(p[1])])

#sigma acts as matrix in basis B on a vector v
def act_vect(sig,v,B=identity_matrix(N)): return (B.inverse()*sig.matrix()*B)*v

#permutation in Sigma_N which induced from perm in Sigma_n
def embed(sigma,n): return perm_from_sort([act(sigma,p) for p in pairs(n)])

#find permutation which orders a list L	
def perm_from_sort(L): return Permutation([pair[0] for pair in sorted(enumerate(L, 1), key=lambda x: x[1])]).to_cycles()

#image of generators of Sigma_n under embedding into Sigma_N
def embed_gens(n): return [embed(gen,n) for gen in SymmetricGroup(n).gens()]

In [9]:
#fundamental domain given arbitrary distinct vector l and basis B
def fund_domain(center=[i for i in range(N)],basis=identity_matrix(N),br=QQ,group=MatrixGroup(identity_matrix(N))):

    #augmented matrix for half-plane ineqs
    A = [ [center[j] - act_vect(group[i],vector(center),basis)[j] for j in range(N)] for i in range(group.order())]
    b_1 = group.order()*[0]
    bA = matrix(br,b_1).transpose().augment(matrix(br,A))

    #augmented matrix for positivity ineqs
    pos = basis.inverse().transpose()
    b_2 = [0 for i in range(N)]
    bPos = matrix(br,b_2).transpose().augment(pos)

    #augmented matrix for fund. domain
    aug_l = bA.stack(bPos)
    
    #find convex polyhedral region, i.e. intersection of all ineqs
    poly = Polyhedron(ieqs=aug_l,base_ring=br)

    return([aug_l,poly])

In [10]:
#slice the fundamental domain with the plane x_1 + ... + x_N = 1
def cross_section(region,slice_dir=[1 for i in range(N)],chi=1,br=None):
    if br==None: br=region.base_ring()
    return Polyhedron(ieqs=region.inequalities(),eqns=[[-1]+slice_dir],base_ring=br)

In [20]:
#find fixed points under G in polyhedron F to include as new vertices
#find intersection of fixed point subspace with F as polyhedron, and return those vertices
def fixed_verts(F,br=QQ):
    new_verts=set()
    for g in G:
        #get equations for fixed point subspace
        A=g.matrix() #matrix associated to group element g
        B=A-identity_matrix(N) #eqns defining fixed pt subspace are (A-Id)x == 0
        b = N*[0] #0 vector
        bB = matrix(br,b).transpose().augment(matrix(br,B))#form augmented matrix for equations defining subspace
        #get equations for intersection
        eqns=matrix(br,[list(eq) for eq in F.equations()]) #list of equations defining F
        eqns=eqns.stack(bB) #augmented matrix including fixed point subspace equations
        #get list of inequalities for fundamental domain as convex polyhedron
        ieqs=matrix(br,[list(ieq) for ieq in F.inequalities()])
        #form intersection of fund_domain with fixed point subspace
        intersection_F_fixed_pt_subspace=Polyhedron(ieqs=ieqs,eqns=eqns)
        verts=intersection_F_fixed_pt_subspace.vertices()
        for vert in verts:
            new_verts.add(tuple(vert))
    return list(new_verts)

In [12]:
#define the finite group
n = 4
N=binomial(n,2)
G = PermutationGroup(embed_gens(n)) #symmetric group \sigma_n as a subgroup of \sigma_N, with 2 generators

In [13]:
#compute fundamental domain to get list of vertices
F=fund_domain(group=G)
F2=cross_section(F[1]); F2.vertices()

(A vertex at (1/2, 0, 0, 0, 0, 1/2),
 A vertex at (0, 1/2, 0, 0, 1/2, 0),
 A vertex at (0, 0, 0, 0, 1/2, 1/2),
 A vertex at (0, 0, 0, 1/3, 1/3, 1/3),
 A vertex at (0, 0, 1/2, 1/2, 0, 0),
 A vertex at (0, 0, 1/3, 0, 1/3, 1/3),
 A vertex at (0, 0, 0, 0, 0, 1))

In [205]:
vertices=fixed_verts(F2,QQ); vertices

[(0, 1/4, 1/4, 1/4, 1/4, 0),
 (1/2, 0, 0, 0, 0, 1/2),
 (0, 0, 1/3, 0, 1/3, 1/3),
 (1/4, 0, 1/4, 1/4, 0, 1/4),
 (1/4, 1/4, 0, 0, 1/4, 1/4),
 (0, 0, 0, 1/3, 1/3, 1/3),
 (0, 0, 1/2, 1/2, 0, 0),
 (0, 0, 0, 0, 0, 1),
 (1/6, 1/6, 1/6, 1/6, 1/6, 1/6),
 (0, 0, 0, 0, 1/2, 1/2),
 (0, 1/2, 0, 0, 1/2, 0)]

In [45]:
#sigma acts as matrix in basis B on a vector v
def act(g,v,basis=identity_matrix(N)): return (basis.inverse()*g.matrix()*basis)*vector(v)

In [115]:
#check if two faces (defined by a set of vertex indices) are glued by checking if there
#exists g in G that maps one vertex set to the other
def vertices_glued(i,j,G,vertices):
    for g in G:
        g_vi = act(g,vertices[i])
        if g_vi == vector(vertices[j]):
            return True
    return False

In [127]:
#find classes of vertices which are glued
glued_verts = []
for i in range(len(vertices)):
    found = False
    for equiv_class in glued_verts:
        if len(equiv_class) >= 1:
            if vertices_glued(i,equiv_class[0],G,vertices):
                equiv_class.append(i)
                found=True
    if not found:
        glued_verts.append([i])

In [126]:
glued_verts

[[0, 3, 4], [1, 6, 10], [2], [5], [7], [8], [9]]

In [338]:
def faces_glued(face1,face2,G,vertices):
    for g in G:
        g_face1 = {vertices.index(tuple(act(g,vertices[i]))) if tuple(act(g,vertices[i])) in vertices else i for i in face1}
        if set(g_face1) == set(face2):
            return True
    return False

In [319]:
def gluing_elmts(face1,face2,G,vertices):
    elmts = []
    trivial = True
    for g in G:
        #act on vertices defining face
        g_face1 = [vertices.index(tuple(act(g,vertices[i]))) if tuple(act(g,vertices[i])) in vertices else i for i in face1]
        #if vertex sets match, add element to list
        if set(g_face1) == set(face2):
            elmts.append(g)
            #check if the gluing is non-trivial
            if [g_face1.index(i) for i in face1] != [i for i in range(len(face1))]:
                trivial = False
    return (elmts,trivial)

In [333]:
#toss out any facets with non-trivial self-gluings such as [0,3,4] or [1,6]
#build list of faces for each dim
import itertools
one_faces=[result for result in itertools.combinations(range(len(vertices)), 2) if gluing_elmts(result,result,G,vertices)[1]]
print(one_faces)

[(0, 1), (0, 2), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 2), (1, 3), (1, 4), (1, 5), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


In [395]:
#compute boundary of each face
#replace vertices with glued_verts, keep track of orientation
def face_boundary(face):
    boundary = []
    for i in face:
        face_remove_i = list(face)
        face_remove_i.remove(i)
        boundary.append(face_remove_i)
    return boundary